# Tutorial: Enhanced Canonical Health Economic Analysis Code

This Jupyter Notebook provides a step-by-step guide to using the enhanced canonical health economic analysis codebase, demonstrating its new features and functionalities.

## 1. Introduction

This codebase integrates a comprehensive suite of tools for health economic evaluation, addressing various aspects of cost-effectiveness analysis (CEA), value of information (VOI), discrete choice experiments (DCE), and more. It aims to provide a robust and transparent framework for decision-making in health technology assessment.

## 2. Setup

First, ensure you have the necessary Python packages installed. You can install them using `pip` from the `requirements.txt` file in the project's root directory.

```bash
pip install -r requirements.txt
```

Now, let's import the necessary modules from our `src` directory.


In [ ]:
import os
import sys

# Add the src directory to the path to allow imports
sys.path.insert(0, os.path.abspath('../src'))

import numpy as np
from bia_model import bia_to_markdown_table, project_bia
from cea_model_core import run_cea
from cluster_analysis import ClusterAnalysis
from discordance_analysis import calculate_decision_discordance
from dsa_analysis import perform_comprehensive_two_way_dsa, perform_three_way_dsa
from main_analysis import (
    get_hepatitis_c_parameters,
    get_hpv_parameters,
    get_smoking_cessation_parameters,
    main,
)
from plotting import *
from reporting import generate_comprehensive_report
from threshold_analysis import run_threshold_analysis
from value_of_information import ProbabilisticSensitivityAnalysis

## 3. Running the Main Analysis

The `main` function in `main_analysis.py` orchestrates all analyses. Running it will execute all the integrated features and generate outputs.


In [ ]:
# Run the full analysis
results = main()
print("\nMain analysis completed. Results are stored in the 'results' dictionary and 'output/' directory.")

## 4. Exploring Individual Features

Let's explore some of the key features individually with examples.

### 4.1 Corrected CEA Calculations

The `run_cea` function from `cea_model_core.py` performs the core cost-effectiveness analysis.

In [ ]:
hpv_params = get_hpv_parameters()

hs_results = run_cea(hpv_params, perspective='health_system')
soc_results = run_cea(hpv_params, perspective='societal')

print("Health System Perspective:", hs_results)
print("Societal Perspective:", soc_results)

### 4.2 Discordance Analysis

Quantify the decision discordance between health system and societal perspectives.

In [ ]:
discordance_results = calculate_decision_discordance("HPV Vaccination", hpv_params)
print("Discordance Results:", discordance_results)

### 4.3 Threshold Analysis

Identify parameter values at which decisions would switch.

In [ ]:
parameter_ranges = {
    'new_treatment_cost_multiplier': np.linspace(0.5, 1.5, 5) # Example: vary new treatment cost from 50% to 150%
}
threshold_results = run_threshold_analysis("HPV Vaccination", hpv_params, parameter_ranges)
print("Threshold Analysis Results for cost multiplier:\n", threshold_results['new_treatment_cost_multiplier'])

### 4.4 Deterministic Sensitivity Analysis (DSA)

Perform two-way and three-way DSA and visualize the results.

In [ ]:
interventions = {
    'HPV Vaccination': get_hpv_parameters(),
    'Smoking Cessation': get_smoking_cessation_parameters(),
    'Hepatitis C Therapy': get_hepatitis_c_parameters()
}

dsa_results_2_way = perform_comprehensive_two_way_dsa(interventions, wtp_threshold=50000, n_points=5)
plot_two_way_dsa_heatmaps(dsa_results_2_way, output_dir="../output/figures_tutorial")
print("2-Way DSA plots generated in ../output/figures_tutorial/")

dsa_results_3_way = perform_three_way_dsa(interventions, wtp_threshold=50000, n_points=5)
plot_three_way_dsa_3d(dsa_results_3_way, output_dir="../output/figures_tutorial")
print("3-Way DSA plots generated in ../output/figures_tutorial/")

### 4.5 Budget Impact Analysis (BIA)

Project the financial consequences of adopting new interventions.

In [ ]:
bia_params = {
    'population_size': 100000,
    'eligible_prop': 0.1,
    'uptake_by_year': [0.1, 0.2, 0.3, 0.4, 0.5],
    'cost_per_patient': hpv_params['costs']['health_system']['new_treatment'][0],
    'offset_cost_per_patient': hpv_params['costs']['health_system']['standard_care'][0],
}
bia_df = project_bia(**bia_params)
print("BIA Results:\n", bia_df)
print("\nBIA Markdown Table:\n", bia_to_markdown_table(bia_df))

### 4.6 Cluster Analysis

Identify intervention archetypes using clustering techniques.

In [ ]:
all_probabilistic_results = {}
for name, params in interventions.items():
    psa = ProbabilisticSensitivityAnalysis(run_cea, params, wtp_threshold=50000)
    all_probabilistic_results[name] = psa.run_psa(n_samples=100) # Small sample for tutorial

cluster_analyzer = ClusterAnalysis(all_probabilistic_results, interventions)
cluster_results_hpv = cluster_analyzer.perform_clustering("HPV Vaccination", n_clusters_range=range(2,3))
plot_cluster_analysis({"HPV Vaccination": cluster_results_hpv}, output_dir="../output/figures_tutorial")
print("Cluster Analysis plots generated in ../output/figures_tutorial/")

### 4.7 Automated Reporting

Generate comprehensive Markdown reports for each intervention.

In [ ]:
report = generate_comprehensive_report("HPV Vaccination", hpv_params)
print("\nComprehensive Report for HPV Vaccination:\n", report)
with open("../output/tutorial_hpv_report.md", "w") as f:
    f.write(report)
print("Report saved to ../output/tutorial_hpv_report.md")

## 5. Conclusion

This tutorial has demonstrated the core functionalities of the enhanced health economic analysis codebase. Feel free to modify the parameters, explore different interventions, and experiment with the various analytical and visualization tools provided.